In [1]:
pip install requests 


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\nikol\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
pip install schedule

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: C:\Users\nikol\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
class MagneticStormMonitor:
    def __init__(self, db_path: str = "../app/magnetic_data.db"):  
        self.db_path = db_path
import requests
import json
from datetime import datetime, timedelta
import sqlite3
import time
import schedule
import logging
from typing import List, Dict, Optional

# логи
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('magnetic_storm_monitor.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class MagneticStormMonitor:
    def __init__(self, db_path: str = "magnetic_data.db"):
        self.db_path = db_path
        # NOAA SWPC API 
        self.forecast_url = 'https://services.swpc.noaa.gov/products/noaa-planetary-k-index-forecast.json'
        self.init_database()
    
    def init_database(self):
        """Инициализация базы данных"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Таблица для прогнозов на 3 дня
            cursor.execute('''
                CREATE TABLE IF NOT EXISTS kp_forecasts_3day (
                    id INTEGER PRIMARY KEY AUTOINCREMENT,
                    forecast_timestamp DATETIME UNIQUE,
                    kp_index REAL,
                    forecast_period TEXT,
                    created_at DATETIME DEFAULT CURRENT_TIMESTAMP
                )
            ''')
            
            # Индекс для быстрого поиска
            cursor.execute('CREATE INDEX IF NOT EXISTS idx_forecast_timestamp ON kp_forecasts_3day(forecast_timestamp)')
            
            conn.commit()
            conn.close()
            logger.info("База данных инициализирована успешно")
            
        except sqlite3.Error as e:
            logger.error(f"Ошибка инициализации базы данных: {e}")
    
    def round_kp_index(self, kp_value: float) -> int:
        """Округление Kp-индекса по математическим правилам"""
        return round(kp_value)
    
    def fetch_forecast_data(self) -> Optional[dict]:
        """Загрузка данных прогноза из NOAA API"""
        try:
            logger.info("Загрузка данных прогноза из NOAA API...")
            response = requests.get(self.forecast_url, timeout=30)
            response.raise_for_status()
            
            data = response.json()
            logger.info(f"Успешно загружено {len(data)} записей прогноза")
            return data
            
        except requests.exceptions.RequestException as e:
            logger.error(f"Ошибка при загрузке данных прогноза: {e}")
            return None
        except json.JSONDecodeError as e:
            logger.error(f"Ошибка парсинга JSON прогноза: {e}")
            return None
    
    def parse_forecast_data(self, raw_data: List) -> List[Dict]:
        """Парсинг данных прогноза"""
        parsed_data = []
        
        if not raw_data or len(raw_data) < 2:
            logger.warning("Нет данных прогноза для парсинга")
            return parsed_data
        
      
        useful_data = raw_data[1:]
        
        for row in useful_data:
            try:
                if len(row) < 2:
                    continue
                    
                time_str = row[0]
                kp_str = row[1]
                
                # Парсим timestamp прогноза
                if '.' in time_str:
                    timestamp = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S.%f")
                else:
                    timestamp = datetime.strptime(time_str, "%Y-%m-%d %H:%M:%S")
                
                kp_index = float(kp_str)
                rounded_kp = self.round_kp_index(kp_index)
                
                # Определяем период прогноза
                hour = timestamp.hour
                if 6 <= hour < 12:
                    period = "утро"
                elif 12 <= hour < 18:
                    period = "день"
                elif 18 <= hour < 24:
                    period = "вечер"
                else:
                    period = "ночь"
                
                parsed_data.append({
                    "forecast_timestamp": timestamp,
                    "kp_index": rounded_kp,
                    "forecast_period": period
                })
                
            except (ValueError, IndexError) as e:
                logger.warning(f"Ошибка парсинга прогноза {row}: {e}")
                continue
        
        logger.info(f"Успешно распаршено {len(parsed_data)} прогнозов")
        return parsed_data
    
    def save_forecasts_to_database(self, data: List[Dict]):
        """Сохранение прогнозов в базу данных"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            saved_count = 0
            for record in data:
                try:
                    cursor.execute('''
                        INSERT OR REPLACE INTO kp_forecasts_3day 
                        (forecast_timestamp, kp_index, forecast_period)
                        VALUES (?, ?, ?)
                    ''', (record['forecast_timestamp'], record['kp_index'], 
                          record['forecast_period']))
                    
                    saved_count += 1
                    
                except sqlite3.Error as e:
                    logger.warning(f"Ошибка сохранения прогноза: {e}")
                    continue
            
            conn.commit()
            conn.close() 
            logger.info(f"Сохранено {saved_count} прогнозов в базу данных")
            
        except sqlite3.Error as e:
            logger.error(f"Ошибка работы с базой данных: {e}")
    
    def get_current_kp_status(self) -> Dict:
        """Получение текущего статуса на основе ближайшего прогноза"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Берем ближайший прогноз к текущему времени
            cursor.execute('''
                SELECT forecast_timestamp, kp_index 
                FROM kp_forecasts_3day 
                WHERE forecast_timestamp >= datetime('now', '-3 hours')
                ORDER BY ABS(strftime('%s', forecast_timestamp) - strftime('%s', 'now')) 
                LIMIT 1
            ''')
            
            current_forecast = cursor.fetchone()
            conn.close()
            
            status = {
                'last_update': datetime.now().strftime("%d.%m.%Y %H:%M"),
                'current_kp': None,
                'storm_level': 'Спокойное',
                'storm_level_code': 'quiet',
                'data_source': 'прогноз'
            }
            
            if current_forecast:
                kp = current_forecast[1]
                status['current_kp'] = {
                    'timestamp': current_forecast[0],
                    'kp_index': kp
                }
                
                # Определение уровня магнитной бури
                if kp >= 9:
                    status['storm_level'] = 'Экстремальная буря'
                    status['storm_level_code'] = 'extreme'
                elif kp >= 8:
                    status['storm_level'] = 'Очень сильная буря'
                    status['storm_level_code'] = 'severe'
                elif kp >= 7:
                    status['storm_level'] = 'Сильная буря'
                    status['storm_level_code'] = 'strong'
                elif kp >= 6:
                    status['storm_level'] = 'Умеренная буря'
                    status['storm_level_code'] = 'moderate'
                elif kp >= 5:
                    status['storm_level'] = 'Небольшая буря'
                    status['storm_level_code'] = 'minor'
                else:
                    status['storm_level'] = 'Спокойное'
                    status['storm_level_code'] = 'quiet'
            
            return status
            
        except sqlite3.Error as e:
            logger.error(f"Ошибка получения текущего статуса: {e}")
            return {}
    
    def get_3day_forecast_summary(self) -> Dict:
        """Получение сводки прогноза на 3 дня"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            
            # Прогнозы на ближайшие 3 дня
            cursor.execute('''
                SELECT 
                    forecast_timestamp,
                    kp_index,
                    forecast_period
                FROM kp_forecasts_3day 
                WHERE forecast_timestamp >= datetime('now')
                AND forecast_timestamp <= datetime('now', '+3 days')
                ORDER BY forecast_timestamp
            ''')
            
            forecasts = cursor.fetchall()
            conn.close()
            
            # Группировка по дням
            daily_forecasts = {}
            for forecast in forecasts:
                timestamp = datetime.fromisoformat(forecast[0])
                date_str = timestamp.strftime("%Y-%m-%d")
                kp_index = forecast[1]
                period = forecast[2]
                
                if date_str not in daily_forecasts:
                    daily_forecasts[date_str] = {
                        'date': timestamp.strftime("%d.%m.%Y"),
                        'day_name': self.get_russian_day_name(timestamp),
                        'periods': [],
                        'max_kp': 0,
                        'storm_risk': 'Низкий'
                    }
                
                daily_forecasts[date_str]['periods'].append({
                    'period': period,
                    'kp_index': kp_index,
                    'time': timestamp.strftime("%H:%M")
                })
                
                # Обновляем максимальный Kp-индекс
                if kp_index > daily_forecasts[date_str]['max_kp']:
                    daily_forecasts[date_str]['max_kp'] = kp_index
            
            # Определяем риск магнитных бурь для каждого дня
            for day in daily_forecasts.values():
                max_kp = day['max_kp']
                if max_kp >= 6:
                    day['storm_risk'] = 'Высокий'
                elif max_kp >= 5:
                    day['storm_risk'] = 'Средний'
                else:
                    day['storm_risk'] = 'Низкий'
            
            return {
                'forecast_generated': datetime.now().strftime("%d.%m.%Y %H:%M"),
                'daily_forecasts': list(daily_forecasts.values())
            }
            
        except sqlite3.Error as e:
            logger.error(f"Ошибка получения прогноза: {e}")
            return {}
    
    def get_russian_day_name(self, date: datetime) -> str:
        """Получение русского названия дня недели"""
        days = {
            'Monday': 'Понедельник',
            'Tuesday': 'Вторник',
            'Wednesday': 'Среда',
            'Thursday': 'Четверг',
            'Friday': 'Пятница',
            'Saturday': 'Суббота',
            'Sunday': 'Воскресенье'
        }
        english_day = date.strftime("%A")
        return days.get(english_day, english_day)
    
    def collect_forecast_data(self):
        """Сбор данных прогноза"""
        logger.info("Начало сбора данных прогноза...")
        
        # Сбор прогнозов
        forecast_data = self.fetch_forecast_data()
        if forecast_data:
            parsed_forecast = self.parse_forecast_data(forecast_data)
            self.save_forecasts_to_database(parsed_forecast)
        
        # Логирование текущего статуса и прогноза
        status = self.get_current_kp_status()
        forecast_summary = self.get_3day_forecast_summary()
        
        logger.info(f"Текущий статус: {status['storm_level']}")
        if status['current_kp']:
            logger.info(f"Текущий Kp-индекс: {status['current_kp']['kp_index']}")
        
        logger.info("Прогноз на 3 дня:")
        for day in forecast_summary.get('daily_forecasts', []):
            logger.info(f"  {day['date']} ({day['day_name']}): макс. Kp={day['max_kp']}, риск бурь: {day['storm_risk']}")
        
        logger.info("Сбор данных прогноза завершен")

def main():
    """Основная функция"""
    monitor = MagneticStormMonitor()
    
    # Немедленный сбор данных
    monitor.collect_forecast_data()
    
    # Настройка регулярного сбора (каждые 3 часа)
    schedule.every(3).hours.do(monitor.collect_forecast_data)
    
    logger.info("Монитор магнитных бурь запущен. Сбор данных каждые 3 часа.")
    
    try:
        while True:
            schedule.run_pending()
            time.sleep(60)  # Проверка каждую минуту
    except KeyboardInterrupt:
        logger.info("Монитор магнитных бурь остановлен")

if __name__ == "__main__":
    main()





2025-12-16 16:11:07,489 - INFO - База данных инициализирована успешно
2025-12-16 16:11:07,489 - INFO - Начало сбора данных прогноза...
2025-12-16 16:11:07,490 - INFO - Загрузка данных прогноза из NOAA API...
2025-12-16 16:11:07,829 - INFO - Успешно загружено 82 записей прогноза
2025-12-16 16:11:07,832 - INFO - Успешно распаршено 81 прогнозов
2025-12-16 16:11:07,842 - INFO - Сохранено 81 прогнозов в базу данных
2025-12-16 16:11:07,845 - INFO - Текущий статус: Спокойное
2025-12-16 16:11:07,846 - INFO - Текущий Kp-индекс: 2.0
2025-12-16 16:11:07,847 - INFO - Прогноз на 3 дня:
2025-12-16 16:11:07,848 - INFO -   16.12.2025 (Вторник): макс. Kp=2.0, риск бурь: Низкий
2025-12-16 16:11:07,848 - INFO -   17.12.2025 (Среда): макс. Kp=4.0, риск бурь: Низкий
2025-12-16 16:11:07,849 - INFO -   18.12.2025 (Четверг): макс. Kp=5.0, риск бурь: Средний
2025-12-16 16:11:07,850 - INFO -   19.12.2025 (Пятница): макс. Kp=4.0, риск бурь: Низкий
2025-12-16 16:11:07,851 - INFO - Сбор данных прогноза завершен
20